## Demo for Random TODOList

- 我的 Random 应该加在 GitHub Action 上面

## refer

- https://pypi.org/project/py-trello/
- https://networkx.org/documentation/stable/tutorial.html

In [1]:
import configparser
import pandas as pd
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [2]:
from trello import TrelloClient

trello_client = TrelloClient(
    api_key=config["defaut"]["api_key"],
    api_secret=config["defaut"]["api_secret"],
    token=config["defaut"]["token"]
)
all_boards = trello_client.list_boards()
last_board = all_boards[0]
print(last_board.name)

Hi-Kanban


In [3]:
from telethon import TelegramClient, events, sync
import socks
import os

SESSION = os.environ.get('TG_SESSION', 'interactive')
API_ID = int(config["telegram"]["tg_api_id"])
API_HASH = config["telegram"]["tg_api_hash"]
PROXY = (socks.SOCKS5, '192.168.31.106', 1080)

if f"{SESSION}.session" in os.listdir():
    os.remove(f"{SESSION}.session")

import sys
import time
client = TelegramClient(SESSION, API_ID, API_HASH, proxy=PROXY)
from IPython.display import clear_output
await client.start()
clear_output()
channel_username = 'niracler_channel'  # your channel

In [4]:
import re
category_str = "Knowledge|Output|ACG|Keep"

def map_to(x):
    weights = re.findall('\((.*?)\)[ ]*\|', x.name)
    weight = int(weights[0]) if len(weights) >= 1 else 5
    categorys = re.findall("^(" + category_str + ")[^(^|.]*", x.name)
    category = categorys[0] if len(categorys) >= 1 else "null"
    name = re.sub("^" + category + "[^|.]*\|", "", x.name).strip()
    return { 
        "name": name,
        "weight": weight, 
        "category": category, 
        "labels": [ label.name for label in x.labels ],
        "id": x.id,
        "title": x.name, 
        "short_url": x.shortUrl
    }

In [5]:
all_boards[0].list_lists()[3].id

'631058bee13e0d048d72c450'

In [6]:
todo_list = last_board.get_list("631058bee13e0d048d72c450")
l = pd.DataFrame(list(map(map_to, todo_list.list_cards())))
l = l.set_index("id")
print(len(l))
print(l.groupby(['category'])['weight'].sum() / l['weight'].sum() * 100)
l[-10:]

16
category
ACG          32.407407
Keep          9.259259
Knowledge    53.703704
Output        4.629630
Name: weight, dtype: float64


,name,weight,category,labels,title,short_url
id,,,,,,
62b57ab6aa6fbe705a821a1d,Forza Horizon 5 | niracler,5,ACG,[],ACG| Forza Horizon 5 | niracler,https://trello.com/c/1zSvVToS
62b57b07016d4d59410fb581,勇者斗恶龙系列 | 勇者斗恶龙：创世小玩家2 破坏神席德与空荡岛 | 勇者斗恶龙11S | ...,5,ACG,[],ACG | 勇者斗恶龙系列 | 勇者斗恶龙：创世小玩家2 破坏神席德与空荡岛 | 勇者斗恶龙...,https://trello.com/c/LjkKkS2l
62b57d8d780dde3647e1ba2d,关于记录生活需要有照片这个事情 | 一天拍10选1 | niracler,5,Output,[],Output | 关于记录生活需要有照片这个事情 | 一天拍10选1 | niracler,https://trello.com/c/Hf8Ujtkp
62b576a357dd3e4a8c5de383,The Legend of Zelda: Breath of the Wild | nira...,10,ACG,[],ACG(10) | The Legend of Zelda: Breath of the W...,https://trello.com/c/FjQNWFZY
62bd1284bb87701d6c0c6d8f,"代码质量，这个我需要深入才行, 多看几本关于代码质量的书籍 | 2本 | niracler",5,Knowledge,[],"Knowledge | 代码质量，这个我需要深入才行, 多看几本关于代码质量的书籍 | 2本...",https://trello.com/c/VxhtAs7u
62b5703983ee6965ea981880,The Witcher 3 in Switch | niracler,5,ACG,[game],ACG | The Witcher 3 in Switch | niracler,https://trello.com/c/EQeRu9aQ
62b57c8f1ed2c520ebd12990,Cyberpunk series of books to read | 2 of these...,5,Knowledge,[Reading Notes],Knowledge| Cyberpunk series of books to read |...,https://trello.com/c/2LFqWi2e
62b57ce1f075270385b0b56b,写作知识补充，阅读之前买的那些关于写作的书 | niracler,10,Knowledge,[],Knowledge(10)| 写作知识补充，阅读之前买的那些关于写作的书 | niracler,https://trello.com/c/YuvRSj8g
62bf92092a6b6d294adf428e,Research on GPT-3 and what OpenAI really has t...,10,Knowledge,[Research],Knowledge(10) | Research on GPT-3 and what Ope...,https://trello.com/c/o5NCBrXu


In [7]:
import random, datetime, time

def rs():
    return random.choices(
        population=list(l.index),
        weights=list(l.weight),
        k=1
    )

e = datetime.datetime.now()
month = datetime.datetime.today().month

for i in range(10):
    time.sleep(3) # Delay for 3 seconds.
    random_id = rs()[0]
    if i < 9:
        print("不是: " + str(l.loc[random_id]['name']))
        
for card in todo_list.list_cards():
    if(card.id == random_id):
        card.change_list("6024cd248ecb43309b5eb2d0")
        
msg = "{month}月份就是这个了(id:{id}):\n\n{title} | {labels}\n\n恭喜恭喜!!!({time})\nurl: {url}\n#random_todolist".format(
    month=month,
    id=random_id,
    url=l.loc[random_id]['short_url'],
    title=l.loc[random_id]['title'],
    labels=l.loc[random_id]['labels'], 
    time=e.strftime("%Y-%m-%d %H:%M:%S")
)
message = await client.send_message(entity=channel_username, message=msg)
await client.pin_message(channel_username, message, notify=True)

print("\n" + msg)

不是: 写作知识补充，阅读之前买的那些关于写作的书 | niracler
不是: Persona 5 Royal | niracler
不是: 代码质量，这个我需要深入才行, 多看几本关于代码质量的书籍 | 2本 | niracler
不是: Forza Horizon 5 | niracler
不是: Research on GPT-3 and what OpenAI really has to offer| niracler
不是: Cyberpunk 2077 | niracler
不是: 关于记录生活需要有照片这个事情 | 一天拍10选1 | niracler
不是: The Legend of Zelda: Breath of the Wild | niracler
不是: Research on GPT-3 and what OpenAI really has to offer| niracler

5月份就是这个了(id:60d4440df4d0c34ac8771106):

Knowledge(10) | 看完《深入浅出nodejs》，并做读书笔记 | ['Learn', 'Reading Notes']

恭喜恭喜!!!(2023-05-01 09:55:55)
url: https://trello.com/c/3jedMhpz
#random_todolist


In [8]:
print("hello world")

hello world
